In [2]:
!pip install yargy

    100% |████████████████████████████████| 51kB 449kB/s ta 0:00:01
  Running setup.py bdist_wheel for intervaltree ... done
  Stored in directory: /home/autumn_mint/.cache/pip/wheels/6b/cf/b0/f7ef2d0f504d26f3e9e70c2369e5725591ccfaf67d528fcbc5
Successfully built intervaltree
You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
#imports
from yargy import *
from docx import *
from yargy.predicates import gram, is_capitalized, dictionary, is_upper, length_eq
from yargy.pipelines import morph_pipeline
import re
import glob
import os

In [19]:
#rules

RPD_task_and_goals=morph_pipeline([
    'цели и задачи освоения',
    'цели освоения',
    'задачи освоения',
    'краткое содержание',
    'краткое описание',
    'аннотация'
])

RPD_education_result=morph_pipeline([
    'планируемый результат обучение'
])
RPD_discipline_link = morph_pipeline([
    'место учебный дисциплина',
    'место дисциплины'
])
RPD_discipline_structure=morph_pipeline([
    'содержание дисциплина'
])
RPD_lecture_theme = morph_pipeline([
    'лекции'
])
RPD_practice_theme = morph_pipeline([
    'практическое занятие',
    'семинар',
    'семинарские занятия'
])
RPD_selfwork_theme = morph_pipeline([
    'самостоятельная работа обучающихся',
    'самостоятельная работа студентов'
])
RPD_literature = morph_pipeline([
    'литература'
])

In [20]:
#parsers
parser_RPD_task_and_goals = Parser(RPD_task_and_goals)
parser_RPD_education_result = Parser(RPD_education_result)
parser_RPD_discipline_link = Parser(RPD_discipline_link)
parser_PRD_discipline_structure = Parser(RPD_discipline_structure)
parser_PRD_lecture_theme = Parser(RPD_lecture_theme)
parser_RPD_practice_theme = Parser(RPD_practice_theme)
parser_RPD_selfwork_theme = Parser(RPD_selfwork_theme)
parser_RPD_literature = Parser(RPD_literature)

In [25]:
#get bold text
def is_bold_paragraph(paragraph):
    for run in paragraph.runs:
        if run.bold:
            return True
            break

In [26]:
#iterator for rpd parts
def iter_headings(paragraphs, parser):
    for paragraph in document.paragraphs:
         if re.match('\d.\d.+', paragraph.text)\
         or paragraph.style.name.startswith('Heading')\
         or paragraph.style.name.startswith('Subtitle')\
         or is_bold_paragraph(paragraph)==True:
                for match in parser.findall(paragraph.text):
                    yield paragraph

In [27]:
#experiments
path = 'test'

for filename in glob.glob(os.path.join(path, '*.docx')):
    document = Document(filename)
    print('Документ: ' + filename + '\n')
    for heading in iter_headings(document.paragraphs, parser_RPD_task_and_goals):
        print ('Цели и задачи    ' + heading.text)
    for heading in iter_headings(document.paragraphs, parser_RPD_education_result):
        print ('Результаты обучения    '+ heading.text)
    for heading in iter_headings(document.paragraphs, parser_RPD_discipline_link):
        print ('Связь дисциплины    ' + heading.text)
    for heading in iter_headings(document.paragraphs, parser_PRD_discipline_structure):
        print ('Структура дисциплины    ' + heading.text)
    for heading in iter_headings(document.paragraphs, parser_PRD_lecture_theme):
        print ('Темы лекций    ' + heading.text)
    for heading in iter_headings(document.paragraphs, parser_RPD_practice_theme):
        print ('Темы практик    ' + heading.text)
    for heading in iter_headings(document.paragraphs, parser_RPD_selfwork_theme):
        print ('Самостоятельая работа    ' + heading.text)
    for heading in iter_headings(document.paragraphs, parser_RPD_literature):
        print ('Литература    ' + heading.text + '\n\n')

Документ: test/1_РПД Иностранный язык.docx

Цели и задачи    Цели и задачи освоения учебной дисциплины
Результаты обучения    Перечень планируемых результатов обучения, соотнесенных с планируемыми результатами освоения образовательной программы
Связь дисциплины    Место учебной дисциплины в структуре образовательной программы
Структура дисциплины    Содержание дисциплины (модуля), структурированное по темам (разделам) с указанием отведенного на них количества академических часов и видов учебных занятий
Структура дисциплины    Таблица 2. Содержание дисциплины (модуля), структурированное по разделам (темам) очной/заочной формы обучения
Самостоятельая работа    Объем дисциплины в зачетных единицах с указанием количества академических часов,  выделенных на контактную работу обучающихся с преподавателем  (по видам занятий) и на самостоятельную работу обучающихся
Самостоятельая работа    Перечень учебно-методического обеспечения для самостоятельной работы обучающихся по дисциплине (модулю)
С